# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [1]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="test-bed-fltk-group16-mb"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



To take a quick anonymous survey, run:
  $ gcloud survey



Scale the default-node-pool up.

In [2]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [3]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1016 14:40:19.522955    8400 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [4]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1016 14:40:28.180148    8423 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Sun Oct 16 14:40:40 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [28]:
# HOMEWORKS
EXPERIMENT_FILE="/home/mattiacs/Documents/Developer/Edu/TUDelft/Year_4/Quarter_1/CS4215-QPECS/homeworks_2/ex6_configs/distributed_1_node.json"

# PROJECT
# EXPERIMENT_FILE="../configs/federated_tasks/example_arrival_config.json"
# EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

# EXPERIMENT_FILE="../project/configs/project_arrival_config_comb_6.json"
# CLUSTER_CONFIG="../project/configs/project_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [29]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1016 15:13:16.167296   11372 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled
W1016 15:13:21.079605   11389 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Sun Oct 16 15:13:23 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [84]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1016 19:21:26.882492   28807 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-16-2022 13:13:31 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-16-2022 13:13:31 root         INFO     Starting in cluster mode.
10-16-2022 13:13:31 root         INFO     Starting with experiment replication: 0 with seed: 42
10-16-2022 13:13:31 root         INFO     Starting as Orchestrator
10-16-2022 13:13:31 root         INFO     Starting Orchestrator, initializing resources....
10-16-2022 13:13:31 root         INFO     Loading in cluster configuration file
10-16-2022 13:13:31 root         INFO     Pointing configuration to in cluster configuration.
10-16-2022 13:13:31 root         INFO     Starting cluster manager
10-16-2022 13:13:31 ClusterManager INFO     Spinning up cluster manag

10-16-2022 13:16:37 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:16:38 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 13:16:38 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 13:16:42 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:16:47 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:16:48 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 13:16:52 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:16:57 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:16:59 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 13:17:02 root      

10-16-2022 13:20:33 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:20:37 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 13:20:38 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:20:43 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:20:47 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 13:20:47 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 13:20:48 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:20:53 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:20:57 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 13:20:58 root      

10-16-2022 13:24:28 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:24:33 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:24:35 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 13:24:38 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:24:43 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:24:45 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 13:24:49 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:24:54 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:24:56 ResourceWatchDog INFO     Fetching pod information of 

10-16-2022 13:28:24 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:28:29 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:28:34 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 13:28:34 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:28:40 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:28:44 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 13:28:45 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:28:50 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 13:28:55 ResourceWatchDog INFO     Fetching pod information of 

...

10-16-2022 15:58:38 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 15:58:43 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 15:58:47 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 15:58:48 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 15:58:53 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 15:58:57 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 15:58:58 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 15:59:03 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 15:59:07 ResourceWatchDog INFO     Fetching pod information of 

10-16-2022 16:02:36 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 16:02:38 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:02:43 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:02:46 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 16:02:48 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:02:53 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:02:56 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 16:02:58 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:03:03 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 

10-16-2022 16:06:34 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:06:35 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 16:06:40 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:06:45 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:06:45 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 16:06:50 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:06:55 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:06:55 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 16:07:00 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 

10-16-2022 16:10:33 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 16:10:33 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 16:10:35 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:10:40 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:10:43 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 16:10:45 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:10:50 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:10:54 ResourceWatchDog INFO     Fetching pod information of cluster...
10-16-2022 16:10:55 root         INFO     Waiting for 5e8eac80-b908-4690-9c2c-4bbbf800d3e6 to complete, 0 pending, 0 arrivals
10-16-2022 16:11:00 root      

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [83]:
# To get logs from learners (example)
kubectl logs -n test trainjob-5e8eac80-b908-4690-9c2c-4bbbf800d3e6-master-0

# To get logs from learners (federated learning)
# kubectl logs -n test trainjob-5e8eac80-b908-4690-9c2c-4bbbf800d3e6-worker-0

W1016 19:21:15.851322   28786 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-16-2022 13:13:39 root         INFO     Loading file config/configuration.fltk.json
10-16-2022 13:13:39 root         INFO     Starting in client mode
10-16-2022 13:13:39 root         INFO     Starting with host=trainjob-5e8eac80-b908-4690-9c2c-4bbbf800d3e6-master-0 and port=23456
10-16-2022 13:13:39 root         INFO     Starting Creating client with 0
10-16-2022 13:13:39 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     Initializing learning client
10-16-2022 13:13:39 root         INFO     Getting net: Nets.fashion_mnist_cnn
10-16-2022 13:13:39 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     Preparing learner model with distributed=False
10-16-2022 13:13:39 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [1,     0] lo

10-16-2022 13:36:31 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [8,   250] loss: 0.392
10-16-2022 13:36:49 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [8,   300] loss: 0.388
10-16-2022 13:37:07 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [8,   350] loss: 0.395
10-16-2022 13:37:24 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [8,   400] loss: 0.384
10-16-2022 13:37:42 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [8,   450] loss: 0.381
10-16-2022 13:38:06 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [9,     0] loss: 0.008
10-16-2022 13:38:22 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [9,    50] loss: 0.383
10-16-2022 13:38:39 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [9,   100] loss: 0.382
10-16-2022 13:38:57 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [9,   150] loss: 0.365
10-16-2022 13:39:15 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [9,   200] loss: 0.395
10-16-2022 13:39:32 

10-16-2022 14:01:37 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [16,   400] loss: 0.342
10-16-2022 14:01:55 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [16,   450] loss: 0.322
10-16-2022 14:02:16 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [17,     0] loss: 0.005
10-16-2022 14:02:35 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [17,    50] loss: 0.312
10-16-2022 14:02:53 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [17,   100] loss: 0.320
10-16-2022 14:03:11 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [17,   150] loss: 0.323
10-16-2022 14:03:29 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [17,   200] loss: 0.326
10-16-2022 14:03:47 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [17,   250] loss: 0.323
10-16-2022 14:04:05 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [17,   300] loss: 0.311
10-16-2022 14:04:23 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [17,   350] loss: 0.328
10-16-2022

10-16-2022 14:27:11 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [25,    50] loss: 0.297
10-16-2022 14:27:29 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [25,   100] loss: 0.299
10-16-2022 14:27:46 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [25,   150] loss: 0.293
10-16-2022 14:28:05 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [25,   200] loss: 0.294
10-16-2022 14:28:24 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [25,   250] loss: 0.299
10-16-2022 14:28:42 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [25,   300] loss: 0.289
10-16-2022 14:29:00 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [25,   350] loss: 0.293
10-16-2022 14:29:18 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [25,   400] loss: 0.283
10-16-2022 14:29:37 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [25,   450] loss: 0.279
10-16-2022 14:30:00 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [26,     0] loss: 0.005
10-16-2022

10-16-2022 14:52:53 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [33,   200] loss: 0.262
10-16-2022 14:53:11 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [33,   250] loss: 0.280
10-16-2022 14:53:29 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [33,   300] loss: 0.272
10-16-2022 14:53:47 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [33,   350] loss: 0.266
10-16-2022 14:54:06 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [33,   400] loss: 0.273
10-16-2022 14:54:23 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [33,   450] loss: 0.276
10-16-2022 14:54:46 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [34,     0] loss: 0.004
10-16-2022 14:55:05 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [34,    50] loss: 0.264
10-16-2022 14:55:23 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [34,   100] loss: 0.270
10-16-2022 14:55:42 Client-0-5e8eac80-b908-4690-9c2c-4bbbf800d3e6 INFO     [34,   150] loss: 0.277
10-16-2022

...

       [  0,   1,   5,   6,   3,   2,   4,   5, 974,   0],
       [  0,   0,   0,   0,   0,   5,   1,  32,   1, 961]])), EpochData(epoch_id=44, num_epochs=81, duration_train=8099649, duration_test=17004, loss_train=0.24600681632757188, accuracy=89.33, loss=23.281899243593216, class_precision=array([0.82890855, 0.99083503, 0.82330097, 0.89328063, 0.827     ,
       0.96513944, 0.72823779, 0.94694695, 0.96719682, 0.95634921]), class_recall=array([0.843, 0.973, 0.848, 0.904, 0.827, 0.969, 0.686, 0.946, 0.973,
       0.964]), confusion_mat=array([[843,   0,  18,  23,   4,   4,  98,   1,   9,   0],
       [  3, 973,   2,  13,   4,   0,   2,   0,   3,   0],
       [ 17,   2, 848,   9,  65,   0,  54,   0,   5,   0],
       [ 20,   5,  12, 904,  24,   0,  34,   0,   1,   0],
       [  1,   1,  73,  32, 827,   0,  65,   0,   1,   0],
       [  0,   0,   0,   0,   0, 969,   0,  18,   1,  12],
       [133,   0,  71,  26,  72,   1, 686,   0,  11,   0],
       [  0,   0,   0,   0,   0,  21,   0, 94

       0.97289157, 0.74070138, 0.93945312, 0.96719682, 0.96969697]), class_recall=array([0.85 , 0.971, 0.838, 0.909, 0.847, 0.969, 0.697, 0.962, 0.973,
       0.96 ]), confusion_mat=array([[850,   0,  16,  23,   6,   3,  90,   1,  11,   0],
       [  4, 971,   2,  14,   4,   0,   2,   0,   3,   0],
       [ 20,   1, 838,  11,  70,   0,  55,   0,   5,   0],
       [ 19,   3,  10, 909,  28,   0,  31,   0,   0,   0],
       [  1,   1,  55,  33, 847,   0,  62,   0,   1,   0],
       [  0,   0,   0,   0,   0, 969,   0,  22,   1,   8],
       [131,   0,  59,  28,  74,   1, 697,   0,  10,   0],
       [  0,   0,   0,   0,   0,  15,   0, 962,   1,  22],
       [  0,   1,   5,   5,   4,   4,   3,   5, 973,   0],
       [  0,   0,   0,   0,   0,   4,   1,  34,   1, 960]])), EpochData(epoch_id=53, num_epochs=81, duration_train=9782247, duration_test=17406, loss_train=0.2429938317835331, accuracy=89.68, loss=22.76017789542675, class_precision=array([0.81998115, 0.99082569, 0.84168337, 0.89130435, 

       0.96 ]), confusion_mat=array([[859,   0,  16,  23,   7,   3,  81,   0,  11,   0],
       [  4, 972,   2,  13,   4,   0,   2,   0,   3,   0],
       [ 17,   2, 835,  11,  79,   0,  51,   0,   5,   0],
       [ 15,   5,  10, 910,  30,   0,  30,   0,   0,   0],
       [  1,   1,  50,  32, 863,   0,  52,   0,   1,   0],
       [  0,   0,   0,   0,   0, 972,   0,  19,   1,   8],
       [131,   0,  61,  30,  86,   1, 682,   0,   9,   0],
       [  0,   0,   0,   0,   0,  16,   0, 962,   1,  21],
       [  0,   1,   5,   5,   5,   4,   1,   5, 974,   0],
       [  0,   0,   0,   0,   0,   5,   1,  33,   1, 960]])), EpochData(epoch_id=61, num_epochs=81, duration_train=11270448, duration_test=17197, loss_train=0.24241795897483825, accuracy=89.76, loss=22.485900312662125, class_precision=array([0.8359841 , 0.99183673, 0.84708249, 0.88987217, 0.82404692,
       0.97291876, 0.73291925, 0.9395122 , 0.9681592 , 0.97163121]), class_recall=array([0.841, 0.972, 0.842, 0.905, 0.843, 0.97 , 0.708,

       [  3, 972,   2,  13,   4,   0,   3,   0,   3,   0],
       [ 18,   0, 847,  10,  63,   0,  58,   0,   4,   0],
       [ 15,   5,  10, 916,  22,   0,  32,   0,   0,   0],
       [  1,   1,  63,  39, 832,   0,  63,   0,   1,   0],
       [  0,   0,   0,   0,   0, 970,   0,  21,   1,   8],
       [127,   0,  65,  31,  67,   0, 700,   0,  10,   0],
       [  0,   0,   0,   0,   0,  14,   0, 965,   1,  20],
       [  0,   1,   5,   5,   5,   3,   1,   5, 975,   0],
       [  0,   0,   0,   0,   0,   4,   1,  34,   1, 960]])), EpochData(epoch_id=69, num_epochs=81, duration_train=12770040, duration_test=15702, loss_train=0.23146543711423873, accuracy=89.86, loss=22.226043447852135, class_precision=array([0.83810464, 0.99284985, 0.83861386, 0.88662791, 0.82970297,
       0.97102897, 0.74019088, 0.94313725, 0.96915423, 0.97163121]), class_recall=array([0.849, 0.972, 0.847, 0.915, 0.838, 0.972, 0.698, 0.962, 0.974,
       0.959]), confusion_mat=array([[849,   0,  16,  23,   6,   3,  94,  

       [ 18,   5,  11, 906,  25,   0,  34,   0,   1,   0],
       [  1,   1,  65,  33, 834,   0,  65,   0,   1,   0],
       [  0,   0,   0,   0,   0, 974,   0,  18,   0,   8],
       [126,   0,  66,  23,  69,   0, 704,   0,  12,   0],
       [  0,   0,   0,   0,   0,  17,   0, 960,   1,  22],
       [  0,   1,   6,   5,   4,   4,   0,   5, 975,   0],
       [  0,   0,   0,   0,   0,   5,   1,  33,   1, 960]])), EpochData(epoch_id=77, num_epochs=81, duration_train=14257046, duration_test=15400, loss_train=0.22855881839990616, accuracy=89.95, loss=22.036042854189873, class_precision=array([0.84075173, 0.99183673, 0.84415584, 0.88629738, 0.8245614 ,
       0.97202797, 0.74128828, 0.94406281, 0.9749499 , 0.97165992]), class_recall=array([0.85 , 0.972, 0.845, 0.912, 0.846, 0.973, 0.702, 0.962, 0.973,
       0.96 ]), confusion_mat=array([[850,   0,  17,  23,   6,   2,  94,   0,   8,   0],
       [  3, 972,   2,  13,   4,   0,   3,   0,   3,   0],
       [ 18,   0, 845,  11,  65,   0,  58,  

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [85]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ../logging

W1016 19:21:50.324098   28900 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1016 19:21:51.286365   28914 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [86]:
helm uninstall -n test experiment-orchestrator

W1016 19:23:53.290003   29023 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [87]:
helm uninstall extractor -n test

W1016 19:24:21.771768   29053 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [88]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1016 19:25:15.840219   29090 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
pytorchjob.kubeflow.org "trainjob-0f924819-3f03-490b-82d1-61b596306018" deleted
pytorchjob.kubeflow.org "trainjob-348609bc-3b32-496b-9850-dea889f4b3ce" deleted
pytorchjob.kubeflow.org "trainjob-5e8eac80-b908-4690-9c2c-4bbbf800d3e6" deleted
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].
